In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [4]:
train = pd.read_csv('train.csv',header=0)
train_pre = pd.read_excel('train_Preprocessing.xlsx',header=0,sheetname='Log Returns')
test_pre = pd.read_excel('train_Preprocessing.xlsx',header=0,sheetname='test_LogReturns')
test = pd.read_csv('test.csv',header=0)
print train_pre.shape,train.shape,test.shape,test_pre.shape

(2999, 115) (3000, 102) (1361, 101) (1361, 104)


In [ ]:
X = train.values[:,1:101]
col = list(np.concatenate((range(1,101),[107,108]),axis=0))
X_tra = train_pre.values[:,col]
X_tst = test_pre.values[:,1:103]
Y = train.values[:,101]
X_test = test.values[:,1:101]

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

clf = LogisticRegression(verbose=0,n_jobs=-1,multi_class='multinomial',solver='lbfgs',max_iter=100,warm_start=True)
scores = cross_validation.cross_val_score(clf, X, Y, cv=10)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.49833887  0.35215947  0.33        0.37333333  0.42666667  0.31333333
  0.48333333  0.47        0.46153846  0.50167224]
Accuracy: 0.42 (+/- 0.14)


In [169]:
scores = cross_validation.cross_val_score(clf, X_tra, Y[:2999,], cv=10)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.54817276  0.56478405  0.48333333  0.56333333  0.55666667  0.55333333
  0.53        0.55852843  0.57859532  0.56856187]
Accuracy: 0.55 (+/- 0.05)


In [50]:
%timeit
from sklearn.ensemble import RandomForestClassifier
recognizer = RandomForestClassifier(200,verbose=0,oob_score=True,n_jobs=-1,warm_start=True)
scores = cross_validation.cross_val_score(recognizer, X, Y, cv=10)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.49833887  0.37209302  0.33333333  0.35333333  0.41333333  0.33        0.47
  0.5         0.42474916  0.50501672]
Accuracy: 0.42 (+/- 0.13)


In [170]:
scores = cross_validation.cross_val_score(recognizer, X_tra, Y[:2999], cv=10)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.49501661  0.5282392   0.45333333  0.61333333  0.55333333  0.51        0.55
  0.5819398   0.57190635  0.46822742]
Accuracy: 0.53 (+/- 0.10)


In [77]:
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from scipy.stats import uniform as sp_rand
from scipy.stats import randint as sp_randint

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

recognizer = RandomForestClassifier(n_estimators=200,verbose=0,n_jobs=-1,warm_start=True)
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(recognizer, param_distributions=param_dist,
                                   n_iter=n_iter_search,cv=5)

random_search.fit(X, Y)
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=7, max_leaf_nodes=None,
            min_samples_leaf=10, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [80]:
recognizer = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=7, max_leaf_nodes=None,
            min_samples_leaf=10, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)
recognizer.fit(X,Y)
Y_test = recognizer.predict(X_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=7, max_leaf_nodes=None,
            min_samples_leaf=10, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [171]:
random_search.fit(X_tra, Y[:2999,])
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_samples_leaf=6, min_samples_split=7,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [173]:
recognizer = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_samples_leaf=6, min_samples_split=7,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)
recognizer.fit(X_tra,Y[:2999])
Y_test = recognizer.predict(X_tst)

In [ ]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 
# Optimize for accuracy since that is the metric used in the Adult Data Set notation

optimized_GBM.fit(X, Y)
optimized_GBM.grid_scores_

/home/mckc/anaconda/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/mckc/anaconda/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/mckc/anaconda/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.vie

In [174]:
submission = np.transpose(np.vstack((np.hstack(('Time',test.values[:,0])),np.hstack(('Y',Y_test)))))
np.savetxt('submission.txt',submission,delimiter=',',fmt='%s')